# Digit Recognizer

My first, highly experimentative attempt at creating a digit recognizer using a neural network with regularization.

## Base Neural Network Class

The NeuralNetwork class is very simple. It initializes and holds the neural network, is capable of evaluating the network at a given point, and executing back propagation on a given set of data. This gives the client class quite a bit of freedom.

In [ ]:
import numpy as np


class NeuralNetwork:
    def __init__(self, network_architecture, regularization_constant=0.0, learning_rate=0.1):
        self.regularization_constant = regularization_constant
        self.learning_rate = learning_rate
        self.num_layers = len(network_architecture)
        self.network_architecture = network_architecture
        # Here, weights[layer][i] gives the list of weights fed into the ith node of the given layer.
        self.weights = self._create_weight_array()
        # Here biases[layer][i] is the bias for the ith node of the given layer.
        self.biases = self._create_node_array()

    def _create_weight_array(self, randomize=True):
        arr = [None]
        for layer in range(1, self.num_layers):
            # We use normalized Xavier initialization.
            if randomize:
                randomizer = np.vectorize(lambda x: (np.random.random() - 0.5) * 2 * np.sqrt(6)
                    / np.sqrt(self.network_architecture[layer - 1] + self.network_architecture[layer]))
                arr.append(randomizer(
                    np.empty((self.network_architecture[layer], self.network_architecture[layer - 1]))
                ))
            else:
                arr.append(np.zeros((self.network_architecture[layer], self.network_architecture[layer - 1])))
        return arr

    def _create_node_array(self, randomize=True):
        arr = [None]
        for layer in range(1, self.num_layers):
            # We use normalized Xavier initialization.
            if randomize:
                randomizer = np.vectorize(lambda x: ((np.random.random() - 0.5) * 2 * np.sqrt(6)
                    / np.sqrt(self.network_architecture[layer - 1] + self.network_architecture[layer])))
                arr.append(randomizer(
                    np.empty((self.network_architecture[layer]))
                ))
            else:
                arr.append(np.zeros((self.network_architecture[layer])))
        return arr

    def compute_output_layers(self, input_layers):
        curr_layers = np.transpose(input_layers)
        for layer in range(1, self.num_layers):
            curr_layers = np.tanh(self.weights[layer] @ curr_layers
                + self.biases[layer].reshape(self.network_architecture[layer], 1))
        return np.transpose(curr_layers)

    def sgd(self, input_pts, output_pts, batch_size=1, num_epochs=1, log=True):
        num_pts = len(input_pts)
        num_batches = int(np.ceil(num_pts / batch_size))
        for epoch in range(num_epochs):
            if log:
                print("Epoch {} / {}".format(epoch + 1, num_epochs))
            order = np.arange(num_pts)
            np.random.shuffle(order)
            for batch in range(num_batches):
                self._back_prop(input_pts[batch * batch_size:(batch + 1) * batch_size],
                                output_pts[batch * batch_size:(batch + 1) * batch_size])
    
    def _back_prop(self, input_pts, output_pts):
        num_datapts = len(input_pts)
        weight_gradient = self._create_weight_array(randomize=False)
        bias_gradient = self._create_node_array(randomize=False)
        for input_layer, output_layer in zip(input_pts, output_pts):
            self._add_gradient(input_layer, output_layer, weight_gradient, bias_gradient)
        for curr_weight_vector, delta_weight in zip(self.weights, weight_gradient):
            if curr_weight_vector is not None:
                curr_weight_vector *= 1 - 2 * self.learning_rate * self.regularization_constant / num_datapts
                curr_weight_vector -= self.learning_rate * delta_weight / num_datapts
        for curr_bias_vector, delta_bias in zip(self.biases, bias_gradient):
            if curr_bias_vector is not None:
                curr_bias_vector *= 1 - 2 * self.learning_rate * self.regularization_constant / num_datapts
                curr_bias_vector -= self.learning_rate * delta_bias / num_datapts

    def _add_gradient(self, input_layer, output_layer, weight_gradient, bias_gradient):
        neuron_values = self._get_all_layers(input_layer)
        node_value_derivative = self._create_node_array(randomize=False)
        node_inner_value_derivative = self._create_node_array(randomize=False)
        for layer in range(self.num_layers - 1, 0, -1):
            curr_neuron_values = neuron_values[layer]
            if layer == self.num_layers - 1:
                # This is the output layer.
                output_layer_size = self.network_architecture[-1]
                node_value_derivative[layer] = 2 / output_layer_size * (curr_neuron_values - output_layer)
            else:
                node_value_derivative[layer] = (np.transpose(self.weights[layer + 1])
                                                @ node_inner_value_derivative[layer + 1])
            node_inner_value_derivative[layer] = ((1 - curr_neuron_values * curr_neuron_values)
                                                  * node_value_derivative[layer])
            curr_layer_size = self.network_architecture[layer]
            prev_layer_size = self.network_architecture[layer - 1]
            curr_inner_deriv = node_inner_value_derivative[layer].reshape((curr_layer_size, 1))
            prev_vals = neuron_values[layer - 1].reshape((1, prev_layer_size))
            weight_gradient[layer] += curr_inner_deriv @ prev_vals
            bias_gradient[layer] += node_inner_value_derivative[layer]

    def _get_all_layers(self, input_layer):
        layers = [input_layer]
        curr_layer = input_layer
        for layer in range(1, self.num_layers):
            curr_layer = np.tanh(self.weights[layer] @ curr_layer + self.biases[layer])
            layers.append(curr_layer)
        return layers


## Validation

In order to find the optimal parameters, we implement validation with a fifth of the training data. I was initially going to use 10-fold cross validation, but it proved to be too slow.

In [ ]:
# Returns a tuple of arrays.
# First element is the array of validation errors after each epoch.
# Second element is the array of validation accuracies after each epoch.


def calc_errors(input_pts, output_pts, network_architecture,
                regularization_constant=0.0, learning_rate=0.1,
                num_epochs=30, batch_size=1, log=True):
    num_pts = len(input_pts)
    num_train_pts = num_pts * 4 // 5
    num_validation_pts = num_pts - num_train_pts
    training_inputs = input_pts[:num_train_pts]
    validation_inputs = input_pts[num_train_pts:]
    training_outputs = output_pts[:num_train_pts]
    validation_outputs = output_pts[num_train_pts:]
    network = NeuralNetwork(
        network_architecture,
        regularization_constant=regularization_constant,
        learning_rate=learning_rate
    )
    errors = np.empty(num_epochs)
    accuracies = np.zeros(num_epochs)
    for epoch in range(num_epochs):
        if log:
            print("Epoch {} / {}:".format(epoch + 1, num_epochs))
        network.sgd(training_inputs, training_outputs,
                    batch_size=batch_size, log=False)
        validation_results = network.compute_output_layers(validation_inputs)
        errors[epoch] = np.average(np.square(validation_results - validation_outputs))
        for validation_out, validation_exp_out in zip(validation_results, validation_outputs):
            accuracies[epoch] += (np.argmax(validation_out) == np.argmax(validation_exp_out))
        accuracies[epoch] /= num_validation_pts
        if log:
            print("Validation Error: " + str(errors[epoch]))
            print("Accuracy: " + str(accuracies[epoch]))
    return errors, accuracies

## Testing Parameters

At this point, we fetch the testing data.

In [ ]:
import pandas as pd

data = pd.read_csv("train.csv")
data

In [ ]:
num_pts = len(data)
input_pts = data.iloc[:, 1:].to_numpy() * 2 / 255 - 1
labels = data.iloc[:, 0].to_numpy()
output_pts = np.full((num_pts, 10), -1)
for image_index, label in enumerate(labels):
    output_pts[image_index,label] = 1

We now graph out validation error and accuracy for a few sample models with different parameters. First, we vary the learning rate.

In [ ]:
import matplotlib.pyplot as plt
for learning_rate in [0.1, 0.2, 0.3, 0.4, 0.5, 1]:
    print("Learning rate: {}".format(learning_rate))
    errors, accuracies = calc_errors(input_pts, output_pts, (784, 64, 64, 10), learning_rate=learning_rate, batch_size=128, num_epochs=5)
    plt.title("Learning rate: {}".format(learning_rate))
    plt.xlabel("Epochs")
    plt.ylabel("Error")
    plt.plot(errors)
    plt.show()
    plt.title("Learning rate: {}".format(learning_rate))
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.plot(accuracies)
    plt.show()

It seems like a learning rate of 0.2 is good. Now, we vary the regularization constant.

In [ ]:
for rc in [0, 1e-4, 1e-3, 1e-2, 1e-1, 2e-1]:
    errors, accuracies = calc_errors(input_pts, output_pts, (784, 64, 64, 10),
                                     learning_rate=0.2, batch_size=128, num_epochs=30,
                                     regularization_constant=rc)
    print("Regularization: {}".format(rc))
    plt.title("Regularization: {}".format(rc))
    plt.xlabel("Epochs")
    plt.ylabel("Error")
    plt.plot(errors)
    plt.show()
    plt.title("Regularization: {}".format(rc))
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.plot(accuracies)
    plt.show()

Regularization doesn't seem to make a difference. We test the final model.

In [ ]:
errors, accuracies = calc_errors(input_pts, output_pts, (784, 64, 64, 10), learning_rate=.2, batch_size=128, num_epochs=150)
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.plot(errors)
plt.show()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(accuracies)
plt.show()

Now, we create our final model for inference.

In [ ]:
final_model = NeuralNetwork((784, 64, 64, 10), learning_rate=0.2)
final_model.sgd(input_pts, output_pts, batch_size=128, num_epochs=100)

Get the test dataset.

In [ ]:
test_data = pd.read_csv("test.csv")
num_test_pts = len(test_data)
input_test_pts = test_data.to_numpy() * 2 / 255 - 1
test_data

Inference.

In [ ]:
generated_output_layers = final_model.compute_output_layers(input_test_pts)
generated_answers = [np.argmax(layer) for layer in generated_output_layers]

Write output.

In [ ]:
output_dataframe = pd.DataFrame({"ImageID" : np.arange(1, num_test_pts + 1),
                                 "Label" : generated_answers})
output_dataframe.to_csv("answers.csv", index=False)